# This tutorial gives an example of calibrating noise in a complex DP mechanism that performs online query release (OQR) with Private Multiplicative Weight (MW) update.

### What does this algorithm do?

It essentially learns a synthetic dataset while answering a long sequence of low-sensitivity queries on the fly.

The answers are either from the synthetic dataset (considered postprocessing) or privately released from the actual dataset.

The decision on which dataset to use is done in a differentially private manner using the sparse vector technique.

For details, check out this lecture: https://sites.cs.ucsb.edu/~yuxiangw/classes/DPCourse-2021Fall/Lectures/lecture4_private_selection_annotated.pdf and the references therein.

### From autodp point of view

The privacy MW algorithm is essentially a composition of $\frac{16\log{|\mathcal{X}|}}{\alpha^2}$ Laplace mechanisms for releassing queries together with the same number of the Above-threshold algorithm.

## In this tutorial, we will do two things differently

1. We will replace the laplace mechanism with the Gaussian mechanism
2. We will choose the parameters of the mechanism numerically to achieve a pre-defined privacy budget.

### First of all, if we know how to set these parameters then the following is how we should implement this mechanism in autodp

In [1]:
from autodp.mechanism_zoo import GaussianMechanism, PureDP_Mechanism
from autodp.transformer_zoo import Composition

sigma = 10.0
eps = 0.1
k = 100

gm = GaussianMechanism(sigma,name='GM')
SVT = PureDP_Mechanism(eps,name='SVT')
compose = Composition()

composed_mech = compose([gm, SVT], [k, k])

# get the eps as a function of delta
composed_mech.get_eps(delta=1e-6)

/usr/local/lib/python3.8/site-packages/scipy/optimize/_optimize.py:2884: RuntimeWarning: invalid value encountered in scalar divide
  w = xb - ((xb - xc) * tmp2 - (xb - xa) * tmp1) / denom


7.7059259328547824

As we can see it was quite easy to create this mechanism object on-the-fly by the mechanism API. In particular, whenever transformers are used, then we can easily spin-up a *Mechanism object* in the runtime, but it doesn't give us a *Mechanissm class definition* that is needed to use our generic calibration tool.  This example illustrates how to convert this into a mechanism class and use our calibator.

In [2]:
def create_complex_mech(sigma,eps, k):
    
    gm = GaussianMechanism(sigma, name='GM1')
    SVT = PureDP_Mechanism(eps=eps, name='SVT')
    compose = Composition()
    composed_mech = compose([gm, SVT],[k,k])
    
    return composed_mech



# next we can create it as a mechanism class, which requires us to inherit the base mechanism class,
#  which we import now

from autodp.autodp_core import Mechanism

class PrivateMW_Mechanism(Mechanism):
    def __init__(self, sigma,eps,k, name="PrivateMW"):
        self.name = name
        self.params={'sigma':sigma,'eps':eps, 'k':k}
        mech = create_complex_mech(sigma, eps, k)
        # The following will set the function representation of the complex mechanism
        # to be the same as that of the mech
        self.set_all_representation(mech)


        
# There are too many parameters to tweak. Let's instead tweak alpha only and assume 

# sigma *(1/n)* C = alpha
# 1/eps * (1/n) * C = alpha
# k = log|X|/ alpha^2

# for ana appropriately chosen parameter C.

n= 10000
C = 10
logX = 20
        
class PrivateMW_Mechanism_alpha(PrivateMW_Mechanism):
    def __init__(self,alpha,name='PrivateMW'):
        PrivateMW_Mechanism.__init__(self,sigma=alpha/C*n,eps=C/alpha/n,k=logX/alpha**2,name=name)





## Now we are ready to calibrate the parameters of the mechanism!

In [3]:

from autodp.calibrator_zoo import eps_delta_calibrator

calibrate = eps_delta_calibrator()

eps_budget = 2.5

delta_budget= 1e-6

pmw_mech = calibrate(PrivateMW_Mechanism_alpha,eps_budget,delta_budget,[0,100])
alpha = pmw_mech.params['sigma']/n*C

print(pmw_mech.name, pmw_mech.params, pmw_mech.get_eps(delta_budget))
print('Choice of alpha = ',alpha)


PrivateMW {'sigma': 110.77138063570763, 'eps': 0.009027602565401678, 'k': 1629.9521615769388} 2.4999707235939663
Choice of alpha =  0.11077138063570763
